# Clojure is Lisp and Lisp is List Processing

Making a list.  This is an S expression with an operator and three arguments:

In [64]:
(list 1, 2, 3)

(1 2 3)

commas are whitespace

In [65]:
(list 1 2 3)

(1 2 3)

function calls are just lists that start with a function

In [66]:
(+ 1 1)

2

list literals

In [67]:
'(+ 1 1)

(+ 1 1)

In [68]:
(type '(+ 1 1))

clojure.lang.PersistentList

In [69]:
(first '(+ 1 1))

+

In [70]:
(type (first '(+ 1 1)))

clojure.lang.Symbol

In [71]:
(type :yo)

clojure.lang.Keyword

In [72]:
(eval '(+ 1 1))

2

In [73]:
(eval (list + 1 1))

2

In [74]:
(type (first (list + 1 1)))

clojure.core$_PLUS_

# Key terms

## Arity

In [75]:
(defn my-fancy-function [arg1 arg2] (println "whoa you just put in" arg1 "and" arg2))
(my-fancy-function "carrots" "potatoes")

whoa you just put in carrots and potatoes


nil

In [76]:
(my-fancy-function "carrots" "potatoes" "beets")

Execution error (ArityException) at cutcake.core/eval6568 (REPL:1).
Wrong number of args (3) passed to: cutcake.core/my-fancy-function


class clojure.lang.ArityException: 

arity is just the count of arguments allowable for a function

## Destructuring

In [77]:
(ns cutcake.core
  (:require [clojure.math :as math]))

(defn pythagoras1 [p1 p2]
    (let [x1 (get p1 0)
          y1 (get p1 1)
          x2 (get p2 0)
          y2 (get p2 1)
          dx (- x1 x2)
          dy (- y1 y2)]
        (math/sqrt (+ (* dx dx) (* dy dy)))))

#'cutcake.core/pythagoras1

In [78]:
(pythagoras1 [1 1] [4 5])

5.0

In [79]:
(defn pythagoras2 [[x1 y1] [x2 y2]]
    (let [dx (- x1 x2)
          dy (- y1 y2)]
        (math/sqrt (+ (* dx dx) (* dy dy)))))

#'cutcake.core/pythagoras2

In [80]:
(pythagoras2 [1 2] [2 3])

1.4142135623730951

What about parts of the input vectors not mentioned in destructured arguments?  Ignored

In [81]:
(pythagoras2 [1 2 3] [2 3 4])

1.4142135623730951

How to refer to the parts of a vector input of some arbitrary size beyond the expected:

In [82]:
(defn pythagoras3 [[x1 y1 & extra-bits1] [x2 y2 & extra-bits2]]
    (let [dx (- x1 x2)
          dy (- y1 y2)]
        (cond 
            (some? extra-bits1) (str "whoa what is this " (str extra-bits1) " on p1?")
            (some? extra-bits2) (str "whoa what is this " (str extra-bits2) " on p2?")
            :else (math/sqrt (+ (* dx dx) (* dy dy))))))

#'cutcake.core/pythagoras3

In [83]:
(pythagoras3 [1 2 3] [2 3])

"whoa what is this (3) on p1?"

In [84]:
(pythagoras3 [1 2] [2 3 4 5 6])

"whoa what is this (4 5 6) on p2?"

The function now knows about the extra bits, but now the extra bits are lists?

In [85]:
(pythagoras3 [1 2] [2 3])

1.4142135623730951

## Homoiconicity

## Persistent data structures and structural sharing
vectors are immutable

In [86]:
(def my-vec (vector 1 2 3))

#'cutcake.core/my-vec

`my-vec` is a reference to a vector

In [87]:
my-vec

[1 2 3]

we can get a new reference `my-vec-2` for a vector with a different value in positon 0

In [88]:
(def my-vec-2 (assoc my-vec 0 4))

#'cutcake.core/my-vec-2

In [89]:
my-vec-2

[4 2 3]

but `my-vec` has not been updated

In [90]:
my-vec

[1 2 3]

# Functional Programming

Clojure was designed with functional programming in mind.  Functional programming features "pure" functions and treats functions as entities that can be passed around by reference.  Pure functions:
* have outputs strictly determined by their inputs
* have no side effects

Functional programming seeks to constrain the usage of assignment.  -- Robert C. Martin "Uncle Bob"

Below are several ways to add numbers in a sequence.  Each deals with the constraint on assignment in a different way and some are better than others.

In [91]:
(+ 1 2 3 4 5 6 7 8 9 10)

55

does the same as above, except it feels fancier:

In [92]:
(apply + (range 1 11))

55

In [93]:
(apply + (range 1 1000000))

499999500000

Recursion a common and is often the most readable (although not in this case):

In [94]:
(defn tricky
    [n up-to-excl]
    (if (>= n up-to-excl) 0
        (+ n (tricky (+ 1 n) up-to-excl))))

#'cutcake.core/tricky

In [95]:
(tricky 1 11)

55

but the form above is flawed, as it is still vulnerable to stack overflow:

In [96]:
(tricky 1 1000000)

Execution error (StackOverflowError) at cutcake.core/tricky (REPL:3).
null


class java.lang.StackOverflowError: 

Clojure can do tail recursion, but you have to tell it to do so.  Often it will require some rethinking of your recursion:

In [97]:
(defn tricky2
    [n running-total up-to-excl]
    (if (>= n up-to-excl) running-total
        (recur (+ n 1) (+ running-total n) up-to-excl)))

#'cutcake.core/tricky2

For (recur) to work, it must be the last statement in your function.  Some special forms like (if) can wrap the whole thing, but many forms will prevent tail recursion if they are the top layer on a recursive call.

In [98]:
(tricky2 1 0 10)

45

In [99]:
(tricky2 1 0 1000000)

499999500000

Note that we had to reformulate tricky2 because (recur) needs to be the last statement in the function.  Here's what happens when we try to (recur) with the earlier formulation:

In [100]:
(defn tricky
    [n up-to-excl]
    (if (>= n up-to-excl) 0
        (+ n (recur (+ 1 n) up-to-excl))))

Syntax error (UnsupportedOperationException) compiling recur at (REPL:4:14).
Can only recur from tail position


class clojure.lang.Compiler$CompilerException: 

The tail recursion and apply approaches above work for most situations, but these still end up requiring clojure to keep all the numbers in the sequence in memory, either as stack frames or as a list of arguments for a single + call.  On the other hand (reduce) can maintain something more like the memory profile we are accustomed to:

In [101]:
(reduce + (range 1 11))

55

This is equivalent to

In [102]:
(+ (+ (+ (+ (+ (+ (+ (+ (+ 1 2) 3) 4) 5) 6) 7) 8) 9) 10)

55

except that each output is rolled forward and the used sequence input can be discarded

In [103]:
(reduce + (range 1 1000000))

499999500000

## The seq abstraction
### three operations required for implemenation of seq and you can apply tons of functions

https://clojure.org/reference/sequences

### (reduce f val coll)
see slide for expansion

The docs are full of very useful and interesting examples.  See https://clojuredocs.org/clojure.core/reduce where this was found:

In [104]:
(reduce
  (fn [primes number]
    (if (some zero? (map (partial mod number) primes))
      primes
      (conj primes number)))
  [2]
  (take 1000 (iterate inc 3)))

[2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541 547 557 563 569 571 577 587 593 599 601 607 613 617 619 631 641 643 647 653 659 661 673 677 683 691 701 709 719 727 733 739 743 751 757 761 769 773 787 797 809 811 821 823 827 829 839 853 857 859 863 877 881 883 887 907 911 919 929 937 941 947 953 967 971 977 983 991 997]

### (map f coll)

In [105]:
(map inc [1 2 3])

(2 3 4)

In [106]:
(map vector ["a" "b" "c"] [1 2 3])

(["a" 1] ["b" 2] ["c" 3])

## Let's take another look at Pythagoras

Given sequence $\alpha$ and sequence $\beta$, representing $p_1$ and $p_2$ respectively

$pythag(\alpha,\beta) = \sqrt{\sum_{k} (\alpha_k - \beta_k)^2}$

where when a given sequence is not defined at $k$ we can interpret it as being 0

### $\alpha_k - \beta_k$

In [107]:
(def p1 [1 2 3])
(def p2 [2 3])
(defn delta-maybe [d1 d2] (- d1 d2))
(delta-maybe (get p1 1) (get p2 1))

-1

In [108]:
(delta-maybe (get p1 2) (get p2 2))

Execution error (NullPointerException) at cutcake.core/delta-maybe (REPL:3).
Cannot invoke "Object.getClass()" because "x" is null


class java.lang.NullPointerException: 

In [109]:
(defn delta-resilient [d1 d2]
    (cond (nil? d1) (* -1 d2)
          (nil? d2) d1
          :else (- d1 d2)))

#'cutcake.core/delta-resilient

In [110]:
(delta-resilient (get p1 2) (get p2 2))

3

### $(\alpha_k - \beta_k)^2$

In [111]:
(defn squared [n] (* n n))
(def delta-squared (comp squared delta-resilient))

#'cutcake.core/delta-squared

In [112]:
(delta-squared 1 4)

9

### $\sum_{k} (\alpha_k - \beta_k)^2$ sum of squares

In [113]:
(defn sum-of-squares [p1 p2]
    (apply + (map 
              #(delta-squared (get p1 %) (get p2 %)) 
              (range (max (count p1) (count p2))))))

#'cutcake.core/sum-of-squares

In [114]:
(sum-of-squares [1 2 3] [2 3 4])

3

In [115]:
(sum-of-squares [-1 4] [1 0 1])

21

In [116]:
(+ (squared (- -1 1)) (squared (- 4 0)) (squared (- 0 1)))

21

$pythag(\alpha,\beta) = \sqrt{\sum_{k} (\alpha_k - \beta_k)^2}$

In [117]:
(defn pythag [p1 p2]
    (math/sqrt (sum-of-squares p1 p2)))

#'cutcake.core/pythag

In [118]:
(pythag [0 1 2] [2 1 0])

2.8284271247461903

In [119]:
(math/sqrt (+ (squared (- 0 2)) (squared (- 1 1)) (squared (- 2 0))))

2.8284271247461903

In [120]:
(pythag [1 2 3] [4 5 6 7])

8.717797887081348

In [121]:
(math/sqrt (+ (squared (- 1 4)) (squared (- 2 5)) (squared (- 3 6)) (squared (- 0 7))))

8.717797887081348

# Cut Cake

In [122]:
(ns cutcake.core
  (:require [clojure.data.json :as json]
            [hiccup2.core :as h]
            [clojure.java.io :as io]))

nil

give the cuts possible, eliminating duplicates attributable to symmetry (but not those attributable to order, at the moment)

In [123]:
(defn cut-opts
  ([w h]
   {:LEFTY (cut-opts w h :LEFTY)
    :RITA (cut-opts w h :RITA)})
  ([w h p]
   (cond (= p :LEFTY) (map
                       (fn [n] (list [(- w n) h] [n h]))
                       (range 1 (+ 1 (int (/ w 2)))))
         (= p :RITA) (map
                      (fn [n] (list [w (- h n)] [w n]))
                      (range 1 (+ 1 (int (/ h 2)))))
         :else (throw (Exception. "invalid player")))))

#'cutcake.core/cut-opts

In [124]:
(cut-opts 4 4)

{:LEFTY (([3 4] [1 4]) ([2 4] [2 4])), :RITA (([4 3] [4 1]) ([4 2] [4 2]))}

The board is represented as a list of vectors for each piece of cake left that can be cut by someone (i.e. no 1x1).  The following function iterates through each piece of cake and offers an option for the given player for each way they can cut the given piece.

In [125]:
(defn single-move-results
  ([board player]
   (apply concat (single-move-results '() (first board) (rest board) player)))
  ([left-pile curr right-pile player]
   (if (nil? curr) '()
       (let [w (get curr 0)
             h (get curr 1)
             opts (cut-opts w h player)]
         (cons
          (map
           (fn [m] {:left left-pile :middle m :right right-pile})
           opts)
          (if (= 0 (count right-pile)) '()
              (single-move-results (concat left-pile (list curr))
                                   (first right-pile)
                                   (rest right-pile)
                                   player)))))))

#'cutcake.core/single-move-results

The results are a list of maps with the information about the parts of each option required to help the user understand what happened in each.

In [126]:
(single-move-results '([4 4]) :RITA)

({:left (), :middle ([4 3] [4 1]), :right ()} {:left (), :middle ([4 2] [4 2]), :right ()})

In [127]:
(single-move-results '([2 2] [1 2] [2 1] [5 7]) :RITA)

({:left (), :middle ([2 1] [2 1]), :right ([1 2] [2 1] [5 7])} {:left ([2 2]), :middle ([1 1] [1 1]), :right ([2 1] [5 7])} {:left ([2 2] [1 2] [2 1]), :middle ([5 6] [5 1]), :right ()} {:left ([2 2] [1 2] [2 1]), :middle ([5 5] [5 2]), :right ()} {:left ([2 2] [1 2] [2 1]), :middle ([5 4] [5 3]), :right ()})

In [128]:
(single-move-results '([2 2] [1 2] [2 1] [5 7]) :LEFTY)

({:left (), :middle ([1 2] [1 2]), :right ([1 2] [2 1] [5 7])} {:left ([2 2] [1 2]), :middle ([1 1] [1 1]), :right ([5 7])} {:left ([2 2] [1 2] [2 1]), :middle ([4 7] [1 7]), :right ()} {:left ([2 2] [1 2] [2 1]), :middle ([3 7] [2 7]), :right ()})

In [129]:
(single-move-results '([8 1]) :LEFTY)

({:left (), :middle ([7 1] [1 1]), :right ()} {:left (), :middle ([6 1] [2 1]), :right ()} {:left (), :middle ([5 1] [3 1]), :right ()} {:left (), :middle ([4 1] [4 1]), :right ()})

In [130]:
(single-move-results '([8 1]) :RITA)

()

### some dynamic programming stuff
given a map that contains piece values, evaluate an option's total value, find the best among a list of options

In [131]:
(defn eval-opt
  [opt cache]
  (reduce
   +
   (map #(get cache %) opt)))

(defn eval-opts
  [opts cache player]
  (reduce
   (if (= player :LEFTY) max min)
   (map #(eval-opt % cache) opts)))

#'cutcake.core/eval-opts

given a map that contains piece values and a starting position, identify pieces from the available moves whose value is not yet identified

In [132]:
(defn get-unknown-pieces
  [w h known]
  (let [opts (cut-opts w h)
        all-opts (concat (:RITA opts) (:LEFTY opts))]
    (set
     (filter #(nil? (get known %))
             (apply concat all-opts)))))

#'cutcake.core/get-unknown-pieces

implement the relevant parts of the simplicity rule from Winning Ways to calulate the score for a position based on the players' options

In [133]:
(defn simplicity-rule
  [l r]
  (cond (> l r) (throw (Exception. "l > r"))
        (= l r) 0
        (and (< l 0) (> r 0)) 0
        ;
        ; not checking for fractional cases, but here could be that
        ;
        (and (<= r 0) (<= l 0)) (- r 1)
        (and (>= r 0) (>= l 0)) (+ l 1)
        :else (throw (Exception. "not sure what happened"))))

#'cutcake.core/simplicity-rule

Recursively evaluate positions, maintaining a cache for positions already seen so you do not re-caculate them. The `reduce` allows us to pass the latest cache object from position to position.

## Cutcake Scorer

In [134]:
(defn cutcake
  ([w h]
  (cutcake w h {}))
  ([w h known]
  (cond (= w 1) (assoc known [w h] (* -1 (- h 1)))
        (= h 1) (assoc known [w h] (- w 1))
        :else (let [subs (reduce
                          (fn [cache [ww hh]]
                            (cutcake ww hh cache))
                          known
                          (get-unknown-pieces w h known))
                    l-best (eval-opts (cut-opts w h :LEFTY) subs :LEFTY)
                    r-best (eval-opts (cut-opts w h :RITA) subs :RITA)]
                (assoc subs [w h] (simplicity-rule l-best r-best))))))

#'cutcake.core/cutcake

In [135]:
(cutcake 3 3)

{[2 1] 1, [1 2] -1, [2 2] 0, [1 3] -2, [2 3] 0, [3 1] 2, [3 2] 0, [3 3] 0}

In [136]:
(cutcake 14 14)

{[8 8] 0, [7 6] 0, [8 7] 1, [8 11] 0, [9 8] 0, [7 1] 6, [10 14] 0, [12 12] 0, [8 9] 0, [7 12] -2, [12 6] 2, [13 3] 5, [10 5] 1, [11 9] 0, [11 2] 4, [4 3] 1, [7 11] -1, [2 2] 0, [7 13] -2, [3 9] -3, [13 8] 0, [4 12] -2, [7 7] 0, [2 8] -3, [13 6] 2, [8 4] 1, [2 3] 0, [2 5] -1, [7 2] 2, [10 13] 0, [6 7] 0, [12 13] 0, [12 14] 0, [7 4] 0, [8 3] 3, [3 3] 0, [13 12] 0, [10 9] 0, [5 4] 0, [10 8] 0, [5 10] -1, [6 3] 2, [14 6] 2, [3 4] -1, [11 14] 0, [12 8] 0, [1 12] -11, [12 5] 2, [7 3] 2, [8 6] 1, [12 2] 5, [4 2] 1, [14 13] 0, [7 8] -1, [3 12] -5, [4 14] -2, [13 2] 5, [6 6] 0, [9 6] 1, [1 13] -12, [12 1] 11, [11 13] 0, [13 9] 0, [6 13] -2, [1 9] -8, [8 10] 0, [5 3] 1, [9 9] 0, [13 7] 2, [9 3] 3, [9 12] 0, [4 7] 0, [13 1] 12, [4 10] -1, [3 13] -5, [4 9] -1, [1 10] -9, [2 9] -3, [6 5] 0, [11 11] 0, [5 13] -2, [5 14] -2, [4 11] -1, [4 1] 3, [5 2] 1, [2 13] -5, [4 6] 0, [11 4] 1, [1 4] -3, [10 2] 4, [11 8] 0, [1 11] -10, [5 7] 0, [11 12] 0, [12 7] 2, [8 2] 3, [10 7] 1, [11 10] 0, [1 3] -2, [4 8] -

## Remaining items
* Construct an AI based on the score for each option
* Eliminate duplicates from the options due to ordering
* Represent the state of a game for a user to see visually